# Detect Similar Statements

Let's take a step back for a second. My end goal for all my learning is to build a team of AI Agents to work together and solve problems. When these agents communicate, it will be important to distinguish if they are talking about the same thing.

This notebook is to use prompt engineering and tricky statements to see how well the LLM can detect paraphrasing.

In [1]:
from langchain_community.llms.gpt4all import GPT4All
from langchain.prompts import PromptTemplate

In [2]:
base_prompt = PromptTemplate.from_template((
    "{instructions} "
    "Valid answers are 'yes' or 'no'.\n\n"
    "Statement 1: {statement1}\n"
    "Statement 2: {statement2}\n\n"
    "Answer: "
))

prompts = [
    base_prompt.partial(instructions="Given two statements, check if they paraphrase each other."),
    base_prompt.partial(instructions="Given two statements, check if they imply the same meaning."),
]

In [3]:
def test_all_prompts(model, statements):
    for prompt in prompts:
        print(f"{prompt.partial_variables['instructions'].strip('.')} :: ", (prompt | model).invoke({
            "statement1": statements[0],
            "statement2": statements[1],
        }).strip())

In [4]:
from langchain_community.llms.gpt4all import GPT4All

# mistral-7b download available from the gpt4all website. Use the "Model Explorer"
# https://gpt4all.io/
llm = GPT4All(
    model="../../models/mistral-7b-openorca.Q4_0.gguf",
    max_tokens=512,
)

Let's give it a simple task:

In [5]:
test_all_prompts(llm, [
    "I like to eat apples.",
    "I enjoy eating apples.",
])

Given two statements, check if they paraphrase each other ::  Yes
Given two statements, check if they imply the same meaning ::  Yes


This one is a bit tricky -- because in my opinion they are not paraphrases. Thankfully the second prompt catches it.

In [6]:
test_all_prompts(llm, [
    "I like to eat apples.",
    "Apples are my favorite fruit because they can be red or green.",
])

Given two statements, check if they paraphrase each other ::  Yes
Given two statements, check if they imply the same meaning ::  No


This prompt use case is a bit more complicated because some key words are duplicated between the two statements.

In [7]:
test_all_prompts(llm, [
    "The Taj Mahal is a great tourist attraction in India.",
    "The Taj Mahal is in India.",
])

Given two statements, check if they paraphrase each other ::  No
Given two statements, check if they imply the same meaning ::  No


In [8]:
test_all_prompts(llm, [
    "Thunder is the sound caused by lightning. Children are often scared of thunder, even though it can't hurt them.",
    "The sudden increase in temperature and hence pressure caused by the lightning produces rapid expansion of the air in the path of a lightning bolt. This rapid expansion of air results in a sonic shock wave which can be scary.",
])

Given two statements, check if they paraphrase each other ::  Yes
Given two statements, check if they imply the same meaning ::  yes


In [9]:
test_all_prompts(llm, [
    "I want to increase my income.",
    "I wish I worked at a job that paid more money.",
])

Given two statements, check if they paraphrase each other ::  Yes
Given two statements, check if they imply the same meaning ::  Yes


In [10]:
test_all_prompts(llm, [
    "The box is made of cardboard.",
    "The box is made of wood.",
])

Given two statements, check if they paraphrase each other ::  No
Given two statements, check if they imply the same meaning ::  no
